# Nexus Morphology Integration

## Overview

This notebook provides functionalities of morphology data ingestion to Nexus in the following steps:

1. Configure the Blue Brain Nexus environment
2. Define functions for data ingestion
3. Input necessary metadata
4. Ingest the data and create provenance


## 1: Configure the Blue Brain Nexus environment 

In [139]:
!pip install -U nexus-sdk

Requirement already up-to-date: nexus-sdk in /Users/hulu/anaconda/lib/python3.5/site-packages (0.2.1)
You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [15]:
import nexussdk as nexus
import getpass
import os
from urllib.parse import quote_plus as url_encode
import requests

We will be using in the **production** environment of Blue Brain Nexus hosted in Geneva

In [16]:
deployment = "https://bbp.epfl.ch/nexus/v1"

Provide your **token** below. Your token can be obtained after log-in by clicking on *Copy token* in the top left corner of [Nexus Web](https://sandbox.bluebrainnexus.io/web)

In [17]:
token = getpass.getpass()

········


Configure your environment and token to be used for the nexus python SDK

In [18]:
nexus.config.set_environment(deployment)
nexus.config.set_token(token)

We will be working in the **demo** organization of Blue Brain Nexus

In [19]:
org = "Huanxiang"
project = "seu"

## 2: Predefined Data Ingestion Functions

In [35]:
def create_subject(name, age, strain_label, strain_id=None):
    
    subject_id = "https://bbp.epfl.ch/neurosciencegraph/data/subject." + name
    
    create_subject = False
    
    try:
        response = nexus.resources.fetch(org_label=org, project_label=project, resource_id=subject_id)
    except nexus.HTTPError as e:
        if e.response.status_code == 404:    
            create_subject = True
        else:
            print(e.response.json())
            raise e    
    
    if create_subject:
        if strain_id is None:   
            strain_id = 'http://bbp.epfl.ch/neurosciencegraph/ontologies/speciestaxonomy/' + strain_label
        subject = {
          "@context": "https://bbp.neuroshapes.org",
          "@type": "Subject",
          "@id": subject_id,
          "name": name,
          "species": {
            "@id": "http://purl.obolibrary.org/obo/NCBITaxon_10090",
            "label": "Mus musculus"
          },
          "strain": {
            "@id": strain_id,
            "label": strain_label
          },
          "age": {
            "period": "Post-natal",
            "value": {
              "@value": str(age),
              "@type": "xsd:integer"
            },
            "unitCode": "days"
          }
        }

        try:
            response = nexus.resources.create(data=subject, org_label=org, project_label=project, schema_id='datashapes:subject')
        except nexus.HTTPError as e:
            print(subject)
            print("---")
            print(nexus.tools.pretty_print(e.response.json()))

    return response['@id']

In [36]:
def create_image(name, slice_direction, slice_width, slice_height, number_of_slices, slice_resolution, 
                 slice_interval, subject_id):
    image_id = "https://bbp.epfl.ch/neurosciencegraph/data/imagestack." + name
    
    create_image = False
    
    try:
        response = nexus.resources.fetch(org_label=org, project_label=project, resource_id=image_id)
    except nexus.HTTPError as e:
        if e.response.status_code == 404:    
            create_image = True
        else:
            print(e.response.json())
            raise e    
    
    if create_image:
        image = {
          "@context": "https://bbp.neuroshapes.org",
          "@type": [
            "nsg:ImageStack"
          ],
          "@id": image_id,
          "name": name,
          "sliceDirection": slice_direction,
          "sliceWidth": slice_width,
          "sliceHeight": slice_height,
          "numberOfSlices": number_of_slices,
          "nsg:sliceResolution": {
            "nsg:widthResolution": {
              "@value": slice_width,
              "@type": "xsd:float"
            },
            "nsg:heightResolution": {
              "@value": slice_height,
              "@type": "xsd:float"
            },
            "unitCode": "micron"
          },
          "sliceInterval": {
            "value": {
              "@value": slice_interval,
              "@type": "xsd:float"
            },
            "unitCode": "micron"
          },
          "contribution": {
              "@type": "Contribution",
              "agent": {
                  "@id": "https://www.grid.ac/institutes/grid.33199.31",
                  "label": "Huazhong University of Science and Technology"              
              }
          },
          "wasDerivedFrom": {
              "@id": subject_id
          }
        }

        try:
            response = nexus.resources.create(data=image, org_label=org, project_label=project, 
                                              schema_id='datashapes:imagestack')
        except nexus.HTTPError as e:
            print("---")
            print(nexus.tools.pretty_print(e.response.json()))

    return response['@id']    
    

In [37]:
def create_morphology(name, identifier, image_id, brain_region_label, binary_path, brain_region_id=None):

    # Check if the binary file has been uploaded
    file_id="https://bbp.epfl.ch/neurosciencegraph/data/file.neuronmorphology." + name
    
    create_file = False
    
    try:
        file_response = nexus.files.fetch(org_label=org, project_label=project, file_id=file_id)
    except nexus.HTTPError as e:
        if e.response.status_code == 404:    
            create_file = True
        else:
            print(e.response.json())
            raise e
    
    if create_file:
        url = "{}/files/{}/{}/{}".format(deployment,org,project,url_encode(file_id))
        file_obj = {"file": open(binary_path, "rb")}
        attachment_headers = {}
        attachment_headers["Authorization"] ="Bearer {}".format(token)
        # attachment_headers["Content-Type"] = "text/swc"

        file_response = requests.put(url=url, files=file_obj, headers=attachment_headers).json()
    
    
    morphology_file_meta = {
        "file_name": file_response["_filename"],
        "file_size": file_response["_bytes"],
        "file_id": file_response["@id"],
        "content_url": file_response["_self"],
        "digest_value": file_response["_digest"]["_value"]}    
    
    # Create morphology 
    
    morphology_id = "https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphology." + name
    
    create_morphology = False
    
    try:
        response = nexus.resources.fetch(org_label=org, project_label=project, resource_id=morphology_id)
    except nexus.HTTPError as e:
        if e.response.status_code == 404:    
            create_morphology = True
        else:
            print(e.response.json())
            raise e    
    
    if create_morphology:    
        if brain_region_id is None:
            brain_region_id = "http://bbp.epfl.ch/neurosciencegraph/ontologies/brainregion/" + brain_region_label
        morphology = {
          "@context": "https://bbp.neuroshapes.org",
          "@type": [
            "InVitroWholeBrainReconstructedNeuronMorphology"
          ],
          "@id": morphology_id,
          "name": name,
          "identifier": identifier,
          "contribution": {
              "@type": "Contribution",
              "agent": {
                  "@id": "https://www.grid.ac/institutes/grid.263826.b",
                  "label": "Southeast University"              
              }
          },
          "brainLocation": {
            "brainRegion": {
              "@id": brain_region_id,
              "label": brain_region_label
            }
          },
          "wasDerivedFrom": {
              "@id": image_id
          },
          "distribution":{
               "@type": "DataDownload",
               "contentSize": {
                   "unitCode": "bytes",
                   "value": morphology_file_meta['file_size']
               },
               "contentUrl": morphology_file_meta['content_url'],
               "digest": {
                   "algorithm": "SHA-256",
                   "value": morphology_file_meta['digest_value']
               },
               "encodingFormat": "text/swc",
               "name": morphology_file_meta['file_name']
           }
        }

        try:
            response = nexus.resources.create(data=morphology, org_label=org, project_label=project, 
                                              schema_id='datashapes:invitrowholebrainreconstructedneuronmorphology')
        except nexus.HTTPError as e:
            print("---")
            print(nexus.tools.pretty_print(e.response.json()))

    return response['@id']
    

In [38]:
def create_mtype_annotation(morphology_id, mtype_label, mtype_id=None, comment=""):
    if mtype_id is None:
        mtype_id = "http://bbp.epfl.ch/neurosciencegraph/taxonomies/mtype/" + mtype_label    
    mtype_annotation = {
        "@context": "https://bbp.neuroshapes.org",
        "@type": [
            "nsg:MorphologyAnnotation",
            "nsg:Annotation"
        ],
        "name": 'mtype classification',
        "hasTarget": {
            "@id": morphology_id,
            "@type": "nsg:AnnotationTarget"
        },
        "hasBody": {
           "@id": mtype_id,
           "@type": "nsg:AnnotationBody",
           "skos:note": comment,
           "label": mtype_label
        },
        "contribution": {
          "@type": "Contribution",
          "agent": {
              "@id": "https://www.grid.ac/institutes/grid.263826.b",
              "label": "Southeast University"              
          }
      }
    }
        
    try:
        response = nexus.resources.create(data=mtype_annotation, org_label=org, project_label=project, 
                                          schema_id='datashapes:annotation')
    except nexus.HTTPError as e:
        print("---")
        print(nexus.tools.pretty_print(e.response.json()))

    return response['@id']

In [39]:
def create_reconstruction(image_id, morphology_id):
    reconstruction = {
        "@context": "https://bbp.neuroshapes.org",
        "@type": [
            "prov:Activity",
            "nsg:ReconstructionFromImage"
        ],
        "used": [
        {
          "@id": image_id,
          "@type": [
            "nsg:ImageStack",
            "prov:Entity"
          ]
        }
        ],
        "generated": [
        {
          "@id": morphology_id,
          "@type": [
            "nsg:ReconstructedCell",
            "prov:Entity"
          ]
        }
        ]
    }
    
    try:
        response = nexus.resources.create(data=reconstruction, org_label=org, project_label=project, 
                                          schema_id='datashapes:reconstructionfromimage')
    except nexus.HTTPError as e:
        print("---")
        print(nexus.tools.pretty_print(e.response.json()))

    return response['@id']

In [31]:
def ingest_data(metadata):
    subject_id = create_subject(metadata['subject_name'], metadata['subject_age'], metadata['subject_strain'])
    image_id = create_image(metadata['image_name'], metadata['slice_direction'], metadata['slice_width'],
                            metadata['slice_height'], metadata['number_of_slices'], metadata['slice_resolution'],
                            metadata['slice_interval'], subject_id)
    morphology_id = create_morphology(metadata['morphology_name'], metadata['identifier'], image_id, 
                                      metadata['brain_region_label'], metadata['file_path'])
    reconstruction_id = create_reconstruction(image_id, morphology_id)
    
    if metadata['mtype_label'] is not None and metadata['mtype_label'] is not '':
        mtype_id = create_mtype_annotation(morphology_id, metadata['mtype_label'])
        
    ids = {
        "subject_id": subject_id,
        "image_id": image_id,
        "morphology_id": morphology_id
    }
    
    return ids


## 3: Please input necessary metadata

In [22]:
metadata = {
    # Mouse subject
    "subject_name":  "Ai82;Ai140-373368",
    "subject_age":  54,
    "subject_strain":  "Tnnt1-IRES2-CreERT2",

    # Image stack
    "image_name":  "18455",
    "slice_direction":  "Coronal",
    "slice_width":  38000,
    "slice_height":  20000,
    "number_of_slices":  10000,
    "slice_resolution":  0.2,
    "slice_interval":  1,

    # Morphology
    "morphology_name":  "z3718-x10944-y12140",
    "identifier":  "Agent1",
    "brain_region_label":  "Caudoputamen",
    "mtype_label":  None,
    "file_path":  "/Users/hulu/Desktop/reconstruction data for BBP in 2019.03/18455/z3718-x10944-y12140/18455-3718-x10944-y12140.ano.processed.final.final.swc"
}


## 4: Ingest the data and create provenance

In [34]:
ingest_data(metadata)

{'image_id': 'https://bbp.epfl.ch/neurosciencegraph/data/imagestack.18455',
 'morphology_id': 'https://bbp.epfl.ch/neurosciencegraph/data/neuronmorphology.z3718-x10944-y12140',
 'subject_id': 'https://bbp.epfl.ch/neurosciencegraph/data/subject.Ai82;Ai140-373368'}

## Please send the result Nexus IDs to Patrycja Lurie **<patrycja.lurie@epfl.ch>**